In [548]:
import numpy as np
import re
import nltk
import pandas as pd
#nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [566]:
recommendation = pd.read_csv(r"C:\Users\simba\Desktop\feedbacksFormatted.csv")
X, y = recommendation.Data, recommendation.Target

In [567]:
def sentimentClassification(text):
    blob = TextBlob(text)
    if blob.sentiment.polarity < 0:
        a="Not satisfied"
    elif blob.sentiment.polarity >= 0 and blob.sentiment.polarity <0.6:
        a="Satisfied"
    else:
        a="Very Satisfied"
    return a,blob.sentiment.polarity

In [568]:
def feedback_classification(feedback):
    if feedback=="g":
        a="General"
    elif feedback=="bs":
        a="Behaviour torwards students"
    elif feedback=="ts":
        a="Teaching skills"
    else:
        a="Punctuality"
    return a

In [569]:
def processed_documents(data):
    documentss = []
    from nltk.stem import WordNetLemmatizer
    stemmer = WordNetLemmatizer()
    for sen in range(0, len(data)):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(data[sen]))
    
        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) 
    
        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)
    
        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)
    
        # Converting to Lowercase
        document = document.lower()
    
        # Lemmatization
        document = document.split()

        document = [stemmer.lemmatize(word) for word in document]
        document = ' '.join(document)
    
        documentss.append(document)
    return documentss
    
documents=processed_documents(X)

In [570]:
def vectorization(data):
    vectorizer = CountVectorizer(max_features=500)
    X = vectorizer.fit_transform(data).toarray()
    tfidfconverter = TfidfTransformer()
    X = tfidfconverter.fit_transform(X).toarray()
    tfidfconverter = TfidfVectorizer(max_features=500)
    X = tfidfconverter.fit_transform(data).toarray()
    return X
X=vectorization(documents)

In [571]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [572]:
classifier = RandomForestClassifier(n_estimators=100, random_state=0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [573]:
y_pred = classifier.predict(X_test)

In [575]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print("Accuracy : {}".format(accuracy_score(y_test, y_pred)))

[[ 9  0  0  8]
 [ 1 15  0  2]
 [ 0  0  2  0]
 [ 1  1  0 28]]
              precision    recall  f1-score   support

          bs       0.82      0.53      0.64        17
           g       0.94      0.83      0.88        18
           p       1.00      1.00      1.00         2
          ts       0.74      0.93      0.82        30

    accuracy                           0.81        67
   macro avg       0.87      0.82      0.84        67
weighted avg       0.82      0.81      0.80        67

Accuracy : 0.8059701492537313


# New Feedback


In [577]:
feedback=input("Enter new feedback :\t")
e,y=sentimentClassification(feedback)
feedback=pd.DataFrame([{"Data":feedback,"Target":"dummy"}])
feedback = pd.concat([recommendation , feedback]).reset_index(drop=True)
feedback_X, feedback_y = feedback.Data, feedback.Target
feedback_documents=processed_documents(feedback_X)
feedback_X=vectorization(feedback_documents)
feedbackX_train, feedbackX_test, feedbacky_train= feedback_X[:feedback_X.shape[0]-1,:],feedback_X[0,:],feedback_y.iloc[:feedback_y.size-1]
feedback_model = RandomForestClassifier(n_estimators=1000, random_state=0)
feedback_model.fit(feedbackX_train, feedbacky_train)
feedback_prediction = feedback_model.predict(feedbackX_test.reshape(1,-1))
print("..................................................................")
print("Polarity Score : {}".format(y))
print("Comment  : {}".format(e))
print("Feedback Classification :{}".format(feedback_classification(feedback_prediction)))

Enter new feedback :	The teacher comes early
..................................................................
Polarity Score : 0.1
Comment  : Satisfied
Feedback Classification :General
